In [50]:
%matplotlib inline
import ipywidgets as wg
import matplotlib as mpl
import pandas as pd
import numpy as np
import random
import itertools
from matplotlib import colors
import matplotlib.pyplot as plt
from IPython.display import clear_output, display
import plotly.graph_objects as go
import plotly.express as px
# import helper functions
import voila_helper as vh
import os

np.random.seed(42)

In [72]:
title_html = """
<style>
p {
  margin-bottom: 1.2em;
  line-height: 1.5;
  word-wrap: break-word;
}
p_todo {
  font-size: smaller;
}
</style>

<h2>GeoACT Parameter Visualizations</h2>

<p_todo>TODO List: Last Updated 8/16 8:00 PM</p_todo>

<p><a href="https://github.com/covABM/GeoACT_Classroom">See the Codebase on GitHub</a></p>

<p>LinkedIn profiles: 
<a href="https://www.linkedin.com/in/ilya-zaslavsky-7997ab7/">Dr. Ilya Zaslavsky</a>, 
<a href="https://www.linkedin.com/in/kaushik-ram-ganapathy">Kaushik Ganapathy</a>, and
<a href="https://www.linkedin.com/in/bailey-man/">Bailey Man</a></p>
"""
# Set title widget text
header = wg.HTML(title_html)

In [79]:
### Left Sidebar
# TODO: 
# 1. add all inputs
# 2. add text for all inputs
# 3. add all outputs
# 4. functionality
# 5. model validity
# 6. start paper: use placeholders sections to fill in for equations or figures
# 7. Jupyter Book- host using GithHub Pages
# 8. Integrate Google Sheets
# 9. 

## Text describing user input

## init sim button section for Temporal

## All other input widgets: (for each plot)

# Variants in Colors!!! 4 buttons in 2x2 grid
# hbox([vbox([a, b]), vbox([c, d])])

# 




# plt.ioff()

num_init_label = wg.Label('Initial Samples n=')
num_init_slider = wg.IntSlider(min=1, max=10, step=1, value=3)
init_sample_button = wg.Button(description='Generate')
asymp_label = wg.Label('% Chance Asymptomatic')
asymp_slider = wg.FloatSlider(min=0, max=1, step=0.1, value=0.3)
sample_output = wg.Output()
def on_button_clicked(init_sample_button, sample_output):
    with sample_output:
        return vh.sample_initial(n_initial=sample_output.value,)

init_sample_button.on_click(vh.sample_initial())

inputs = [num_init_label, num_init_slider, asymp_label, asymp_slider, init_sample_button]

sidebar = wg.VBox(inputs, width='auto')

# TODO: make right sidebar observe the left: in particular the onclick event
# TODO: input sections:
# 




In [74]:
### Center

x_symp_count, countdown_curve, x_inf_list, inf_df = vh.sample_initial()
fig_out = wg.Output()
with fig_out:
    ## Plot all outputs in the center column
    
    # Placeholder: Spline Serial Interval from Temporal paper

    # Density Plot: Infectivity (y) vs. Days (x) after symptom onset
    fig, axes = plt.subplots(2)
    ax1, ax2 = axes.flatten()
    ax1.plot(x_symp_count, countdown_curve.pdf(x_symp_count))
    ax1.set_title('Expected days until symptoms appear')
    ax1.set_xlabel('Number of days')
    ax1.set_ylabel('Proportion of infected individuals')

    # Density Plot: Likelihood of Symptom Onset (y) vs. Days (x) after Initial Infection
    # line 1: Histogram
    # line 2: Cumulative Histogram
    ax2.plot(x_inf_list, inf_df.gamma)
    ax2.set_title('Infectivity based on days post-symptom onset')
    ax2.set_xlabel('Number of days after symptom onset')
    ax2.set_ylabel('Initial Infectivity')
    fig.tight_layout()

    # Placeholder: aerosol radius vs distance 


    # Placeholder: virion infectivity fro  m bazant


    # Placeholder: Plot # infections (New and Estimated Active Total) of each variant in SD

    ## Variant Differences in transmission rate:
    # Sources: https://www.cdc.gov/coronavirus/2019-ncov/variants/variant.html
    #        : 
    
    ## Vaccine Efficacy of the different brands vs different variants:
    # Vaccine Options: https://www.yalemedicine.org/news/covid-19-vaccine-comparison
    # Alpha: 
    # Beta: 
    # Gamma: 
    # Delta: https://www.nejm.org/doi/full/10.1056/NEJMoa2108891

    # Placeholder: Plot # deaths (New and Total) from vaccinated vs unvaccinated in SD
    # Source: https://www.sandiegocounty.gov/content/dam/sdc/hhsa/programs/phs/Epidemiology/COVID-19%20Watch.pdf

    # Placeholder: Plot # cases (New and Total) of vaccinated vs unvaccinated in SD

    # Placeholder: Transmission rate vs Time (vax vs unvax)

    # Placeholder: Well-Mixed Room

    # Placeholder: Ventilation rate vs Aerosol Infection / time

    # Placeholder: 

    # Show all plots
    plt.show()
    


In [75]:
# Right Sidebar
# Initial Infection Stats

# When Sample is Generated: return n_initial samples of each distribution



In [76]:
# Footer
description_html = """
    <style>
    p {
        margin-bottom: 1.2em;
        line-height: 1.5;
        word-wrap: break-word;
    }
    </style>

    <p>Special thanks to HDSI, and/or whoever else is paying us, and SD school district I guess </p>
"""
footer = wg.HTML(description_html)

In [77]:
app = wg.AppLayout(
    header=header, 
    left_sidebar=sidebar,
    center=fig_out,
    right_sidebar=None,
    footer=footer,
    pane_widths=[1, 3, 1],
    pane_heights=[1, 5, 1],
)

In [78]:
app

AppLayout(children=(HTML(value='\n<style>\np {\n  margin-bottom: 1.2em;\n  line-height: 1.5;\n  word-wrap: bre…

In [41]:
a = wg.IntSlider(description='a')
b = wg.IntSlider(description='b')
c = wg.IntSlider(description='c')
def f(a, b, c):
    print('{}*{}*{}={}'.format(a, b, c, a*b*c))

out = wg.interactive_output(f, {'a': a, 'b': b, 'c': c})

wg.HBox([wg.VBox([a, b, c]), out])